In [1]:
import pandas as pd
from pathlib import Path

In [2]:
#install tensorflowv2
! pip install tensorflow==2.0.0-rc0 
import tensorflow as tf
print(tf.__version__)


ERROR: Could not find a version that satisfies the requirement tensorflow==2.0.0-rc0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.0.0-rc0


2025-08-18 15:58:26.983594: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755532706.996231       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755532707.000238       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 15:58:27.015748: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.18.0


In [3]:
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestRegressor
import pandas as pd 
import matplotlib.pyplot as plt



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#loading dataset
train_iop_path='/kaggle/input/new-york-city-taxi-fare-prediction/train.csv'
test_iop_path='/kaggle/input/new-york-city-taxi-fare-prediction/test.csv'
dataset_train=pd.read_csv(train_iop_path, nrows=10_000, index_col='key')
dataset_test=pd.read_csv(test_iop_path, nrows=10000, index_col='key')


/kaggle/input/sample_submission.csv
/kaggle/input/test.csv.zip
/kaggle/input/train.csv.zip
/kaggle/input/labels.csv.zip
/kaggle/input/train.csv
/kaggle/input/test.csv
/kaggle/input/labels.csv
/kaggle/input/description.md
/kaggle/input/GCP-Coupons-Instructions.rtf
/kaggle/input/sample_submission.csv.zip
/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv.zip
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv.zip
/kaggle/input/new-york-city-taxi-fare-prediction/labels.csv.zip
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv
/kaggle/input/new-york-city-taxi-fare-prediction/labels.csv
/kaggle/input/new-york-city-taxi-fare-prediction/description.md
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv.zip


In [4]:
print("dataset_train old size", len(dataset_train))

dataset_train = dataset_train[dataset_train.dropoff_longitude != 0]
print("new size", len(dataset_train))
dataset_train.head(5)


dataset_train old size 10000
new size 9810


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
key,,,,,,,
2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [5]:
print("dataset_test old size", len(dataset_test))

dataset_test = dataset_test[dataset_test.dropoff_longitude != 0]
print("new size", len(dataset_test))
dataset_test.head(5)


dataset_test old size 9914
new size 9708


,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
key,,,,,,
2010-10-01 21:26:11.0000001,2010-10-01 21:26:11 UTC,-73.983130,40.761970,-73.994386,40.749236,1
2013-10-06 01:38:00.00000083,2013-10-06 01:38:00 UTC,-73.948505,40.753977,-73.808195,40.731952,2
2012-03-30 19:13:53.0000001,2012-03-30 19:13:53 UTC,-73.973964,40.791979,-73.979018,40.785544,1
2012-02-08 02:57:23.0000001,2012-02-08 02:57:23 UTC,-73.991478,40.738907,-73.907198,40.861572,2
2013-12-13 22:56:00.000000237,2013-12-13 22:56:00 UTC,-73.986281,40.740067,-73.933927,40.856781,2


In [6]:
def get_year(pickup_date):
  return pickup_date.year
def get_month(pickup_date):
  return pickup_date.month
def get_day(pickup_date):
  return pickup_date.day
def get_hour(pickup_date):
  return pickup_date.hour       
     


In [7]:
from datetime import datetime as dt
import warnings

def preparedataset2(datasetname):
    warnings.filterwarnings("ignore")
    datasetname.info()
    datasetname['pickup_datetime']= pd.to_datetime(datasetname['pickup_datetime']) 
    datasetname['pickup_year'] = datasetname.apply(lambda x: get_year(x['pickup_datetime']),axis=1)
    datasetname['pickup_month'] = datasetname.apply(lambda x: get_month(x['pickup_datetime']),axis=1)
    datasetname['pickup_day'] = datasetname.apply(lambda x: get_day(x['pickup_datetime']),axis=1)
    datasetname['pickup_hour'] = datasetname.apply(lambda x: get_hour(x['pickup_datetime']),axis=1)
    datasetname['x_dis'] = (datasetname['dropoff_longitude'] - datasetname['pickup_longitude'])  
    datasetname['y_dis'] = (datasetname['dropoff_latitude'] - datasetname['pickup_latitude']) 
    datasetname['dis'] = ((datasetname['dropoff_longitude'] - datasetname['pickup_longitude'])**2 + (datasetname['dropoff_latitude']-datasetname['pickup_latitude'])**2)**.5 
    datasetname=datasetname.drop(['pickup_datetime'],axis=1)
    datasetname=datasetname.drop(['pickup_longitude'],axis=1)
    datasetname=datasetname.drop(['dropoff_latitude'],axis=1)
    datasetname=datasetname.drop(['dropoff_longitude'],axis=1)
    datasetname=datasetname.drop(['pickup_latitude'],axis=1)

   # datasetname.info()
    return datasetname


In [8]:
#old preprocesing 
from datetime import datetime as dt
import warnings
warnings.filterwarnings("ignore")
def preparedataset(datasetname):
    datasetname['pickup_year']=0
    datasetname['pickup_month']=0
    datasetname['pickup_day']=0
    datasetname['pickup_hour']=0
  #  datasetname['pickup_minute']=0
 #   datasetname['pickup_second']=0
    datasetname['dis'] =0
    datasetname['x_dis']=0
    datasetname['y_dis']=0
    
    datasetname.head()
#print(datetime.strptime(df['pickup_datetime'][0].replace("UTC",''),"%Y-%m-%d %H:%M:%S "))

    for k in range(len(datasetname.index)):
        datetime=dt.strptime(datasetname['pickup_datetime'][k].replace("UTC",''),"%Y-%m-%d %H:%M:%S ")
        datasetname['pickup_year'][k]=datetime.year
        datasetname['pickup_month'][k]=datetime.month
        datasetname['pickup_day'][k]=datetime.day
        datasetname['pickup_hour'][k]=datetime.hour
      # datasetname['pickup_minute'][k]=datetime.minute
      # datasetname['pickup_second'][k]=datetime.second
        
    datasetname['x_dis'] = (datasetname['dropoff_longitude'] - datasetname['pickup_longitude'])  
    datasetname['y_dis'] = (datasetname['dropoff_latitude'] - datasetname['pickup_latitude']) 
    datasetname['dis'] = ((datasetname['dropoff_longitude'] - datasetname['pickup_longitude'])**2 + (datasetname['dropoff_latitude']-datasetname['pickup_latitude'])**2)**.5 
    datasetname=datasetname.drop(['pickup_datetime'],axis=1)
    datasetname=datasetname.drop(['pickup_longitude'],axis=1)
    datasetname=datasetname.drop(['dropoff_latitude'],axis=1)
    datasetname=datasetname.drop(['dropoff_longitude'],axis=1)
    datasetname=datasetname.drop(['pickup_latitude'],axis=1)

    return datasetname


In [9]:
df=preparedataset2(dataset_train)
df.head(5)


<class 'pandas.core.frame.DataFrame'>
Index: 9810 entries, 2009-06-15 17:26:21.0000001 to 2011-07-05 06:48:44.0000002
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare_amount        9810 non-null   float64
 1   pickup_datetime    9810 non-null   object 
 2   pickup_longitude   9810 non-null   float64
 3   pickup_latitude    9810 non-null   float64
 4   dropoff_longitude  9810 non-null   float64
 5   dropoff_latitude   9810 non-null   float64
 6   passenger_count    9810 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 613.1+ KB


,fare_amount,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,x_dis,y_dis,dis
key,,,,,,,,,
2009-06-15 17:26:21.0000001,4.5,1,2009,6,15,17,0.002701,-0.009041,0.009436
2010-01-05 16:52:16.0000002,16.9,1,2010,1,5,16,0.036780,0.070701,0.079696
2011-08-18 00:35:00.00000049,5.7,2,2011,8,18,0,-0.008504,-0.010708,0.013674
2012-04-21 04:30:42.0000001,7.7,1,2012,4,21,4,-0.004437,0.024949,0.025340
2010-03-09 07:51:00.000000135,5.3,1,2010,3,9,7,0.011440,0.015754,0.019470


In [10]:
test_df=preparedataset2(dataset_test)
test_df.head(5)


<class 'pandas.core.frame.DataFrame'>
Index: 9708 entries, 2010-10-01 21:26:11.0000001 to 2014-09-05 11:20:53.0000004
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pickup_datetime    9708 non-null   object 
 1   pickup_longitude   9708 non-null   float64
 2   pickup_latitude    9708 non-null   float64
 3   dropoff_longitude  9708 non-null   float64
 4   dropoff_latitude   9708 non-null   float64
 5   passenger_count    9708 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 530.9+ KB


,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,x_dis,y_dis,dis
key,,,,,,,,
2010-10-01 21:26:11.0000001,1,2010,10,1,21,-0.011256,-0.012734,0.016996
2013-10-06 01:38:00.00000083,2,2013,10,6,1,0.140310,-0.022025,0.142028
2012-03-30 19:13:53.0000001,1,2012,3,30,19,-0.005054,-0.006435,0.008182
2012-02-08 02:57:23.0000001,2,2012,2,8,2,0.084280,0.122665,0.148828
2013-12-13 22:56:00.000000237,2,2013,12,13,22,0.052354,0.116714,0.127918


In [11]:
#spliting the dataset 
from sklearn.model_selection import train_test_split
y = df.fare_amount
X=df.drop('fare_amount',axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y)


In [12]:
#applying XCbossting 
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
result={}
best_istemator=0
best_learing_rate=0
best_mae=100000000000
for lr in [X / 100 for X in range(10,50, 5)]:
    for ns in range(200,701,50):
        #print("n_estimators",ns)
        #print("learning_rate",lr)
        my_model = XGBRegressor(n_estimators=ns, learning_rate=lr,n_jobs=4)
        my_model.fit(X_train, y_train)

        predictions = my_model.predict(X_valid)

        mae=mean_absolute_error(predictions, y_valid)
        if(mae  best_mae):
            best_mae=mae
            best_istemator=ns
            best_learing_rate=lr
            print("better found")
            print(ns , lr, mae)
        result[(ns,lr)]=mae
my_model_2 = XGBRegressor(n_estimators=best_istemator, learning_rate=best_learing_rate, n_jobs=4)
my_model_2.fit(X_train,y_train)
predictions_2 = my_model_2.predict(X_valid)
mae_2 = mean_absolute_error( y_valid, predictions_2) 
# Uncomment to print MAE
print("best_istemator:" , best_istemator)
print("best_learing_rate:" , best_learing_rate)
print("Mean Absolute Error:" , mae_2)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (539299351.py, line 20)

In [13]:
from xgboost import XGBRegressor
my_model_2 = XGBRegressor(n_estimators=700, learning_rate=0.2, n_jobs=4)
my_model_2.fit(X_train,y_train)

test_preds = my_model_2.predict(test_df)

output = pd.DataFrame({'key': test_df.index,
                      'fare_amount': test_preds})
output.to_csv('submission.csv', index=False)
